# Scrape Carvana

* You only need to fetch a single page of vehicle results. e.g., go to https://www.carvana.com/cars and click [Next] page -- Notice it refreshes the inventory without refreshing the page.

* Create a simple MySQL schema for storing these records and include the necessary create table(s) SQL. We are interested in the following fields: vehicle_id, vin, make, model, mileage, price

* How would you extend this to fetch all the vehicle inventory? (Code is not necessary)

In [52]:
# function fetch_carvana_inventory_by_page(int $page_id) : array {
#     // Fetch a single page of vehicles
#     // return an array of vehicle records
#     return [];
# }
# 
# function save_carvana_inventory(array $vehicle) : bool {
#     // Persist a single vehicle record to the database
#     // Return a boolean indicating success or failure
#     return true;
# }
# 
# foreach (fetch_carvana_inventory_by_page(2) as $vehicle) {
#     save_carvana_inventory($vehicle);
# }
# 

### Load imports

In [70]:
from bs4 import BeautifulSoup
import requests
import json
import mysql.connector

### Scrape a singular car webpage

In [54]:
webpage = requests.get("https://www.carvana.com/cars")

In [55]:
soup = BeautifulSoup(webpage.text, "html.parser")

In [56]:
car_script_tags = soup.find_all("script", {"data-react-helmet": True})

In [57]:
len(car_script_tags)

22

In [58]:
car_jsons = [json.loads(car_script_tags[0].contents[0]) for car_script in car_script_tags]

In [59]:
car_jsons[0]

{'@context': 'http://schema.org',
 '@type': 'Vehicle',
 'itemCondition': 'Used',
 'name': '2020 Buick Encore GX',
 'modelDate': '2020',
 'manufacturer': 'Buick',
 'model': 'Encore GX',
 'color': 'Gray',
 'image': '//cdnblob.fastly.carvana.io/2001026852/post-large/normalized/zoomcrop/2001026852-edc-02.jpg',
 'brand': 'Buick',
 'description': 'Used 2020 Buick Encore GX undefined with 273 miles - $26990',
 'mileageFromOdometer': '273',
 'sku': '2001026852',
 'vehicleIdentificationNumber': 'KL4MMBS27LB101541',
 'offers': {'@type': 'Offer',
  'price': '26990',
  'priceCurrency': 'USD',
  'availability': 'http://schema.org/InStock',
  'priceValidUntil': 'January 1, 2030',
  'url': 'https://www.carvana.com/vehicle/1809150'}}

### Connect to Database
We assume that prior to running this python code we have ran the sql script to create the database and insert the table we want to use.

The script can be run via mysql. Install mysql and login with 'mysql -uroot' and run 'source carvana.sql' assuming the script is in the same dir you were in when logging into mysql.

In [60]:
my_db = mysql.connector.connect(
    user='root',
    host='localhost',
    database='carvana',
    buffered=True,
)
cursor = my_db.cursor()

### Load a single page of cars into the database

In [61]:
car = car_jsons[0]

In [62]:
add_car = """INSERT INTO `cars` (`id`,`vehicle_id`,`vin`,`make`,`model`,`mileage`,`price`) VALUES ({},{},'{}','{}','{}',{},{})""".format(
    1,
    car['sku'],
    car['vehicleIdentificationNumber'],
    car['manufacturer'],
    car['model'],
    car['mileageFromOdometer'],
    car['offers']['price']
)

print(add_car)

INSERT INTO `cars` (`id`,`vehicle_id`,`vin`,`make`,`model`,`mileage`,`price`) VALUES (1,2001026852,'KL4MMBS27LB101541','Buick','Encore GX',273,26990)


In [63]:
cursor.execute(add_car)

In [64]:
# save results
my_db.commit()
cursor.close()

True

## Put the above into requested functions

In [65]:
#Fetch a single page of vehicles
#return an array of vehicle records
def fetch_carvana_inventory_by_page(page):
    webpage = requests.get("https://www.carvana.com/cars?email-capture=&page={}".format(page))
    soup = BeautifulSoup(webpage.text, "html.parser")
    car_script_tags = soup.find_all("script", {"data-react-helmet": True})
    car_jsons = [json.loads(car_script.contents[0]) for car_script in car_script_tags]
    return car_jsons

In [66]:
# Persist a single vehicle record to the database
# Return a boolean indicating success or failure
def save_carvana_inventory(vehicle):
    add_vehicle = """INSERT INTO `cars` (`vehicle_id`,`vin`,`make`,`model`,`mileage`,`price`) VALUES ({},'{}','{}','{}',{},{})""".format(
        vehicle['sku'],
        vehicle['vehicleIdentificationNumber'],
        vehicle['manufacturer'],
        vehicle['model'],
        vehicle['mileageFromOdometer'],
        vehicle['offers']['price']
    )
    cursor.execute(add_vehicle)
    return my_db.commit()

In [67]:
# Loops through and saves all cars for each page
def gen_results(page_to_stop_on):
    page_count = 1
    while page_count <= page_to_stop_on:
        try:
            for vehicle in fetch_carvana_inventory_by_page(page_count):
                save_carvana_inventory(vehicle)
        except Exception as e:
            print("Can't get anymore pages.\n{}".format(e))
            break
        page_count += 1

In [68]:
# Connect to local mysql instance
my_db = mysql.connector.connect(
    user='root',
    host='localhost',
    database='carvana',
    buffered=True,
)
cursor = my_db.cursor()

# Get 5 pages worth of results
gen_results(5)



In [73]:
import csv
import pandas

cursor.execute("select * from cars")

# Generate a csv file dump of results
with open("results.csv", 'w', newline='', encoding='utf-8') as csvfile:
    word_writer = csv.writer(csvfile, delimiter=' ', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for row in cursor:
        word_writer.writerow(row[0].split(" "))    
        
        
# Load results into pandas
# Close connection to database when we're done
# my_db.close()

ModuleNotFoundError: No module named 'pandas'